In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [3]:
installs = pd.read_csv("installs.csv.gzip", compression='gzip')

/home/cecilia/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
installs.isnull().sum()

created                    0
application_id             0
ref_type                   0
ref_hash                   0
click_hash            480369
attributed                 0
implicit                   0
device_countrycode         0
device_brand          205068
device_model           26892
session_user_agent     14839
user_agent            150743
event_uuid            378343
kind                  378343
wifi                  186682
trans_id              472578
ip_address                 0
device_language        27577
dtype: int64

In [4]:
installs.shape

(481511, 18)

In [5]:
installs['application_id'].value_counts().head(10)

121    71487
36     64471
210    33672
14     25630
65     23067
68     18762
122    15170
155    13629
226    13469
21     12184
Name: application_id, dtype: int64

In [6]:
installs['ref_type'].value_counts()

1891515180541284343    398906
1494519392962156891     82605
Name: ref_type, dtype: int64

In [7]:
installs['attributed'].value_counts()

False    480369
True       1142
Name: attributed, dtype: int64

In [8]:
installs['implicit'].value_counts()

False    378343
True     103168
Name: implicit, dtype: int64

In [9]:
installs['device_countrycode'].value_counts()

6287817205707153877    481511
Name: device_countrycode, dtype: int64

In [10]:
installs['device_model'].value_counts().head(10)

6.794880e+18    46187
9.186120e+18    13271
5.660121e+18     8562
3.057402e+18     7384
7.346301e+18     6233
2.544397e+18     5635
5.127788e+18     5609
3.017401e+18     5297
9.173807e+18     5242
1.670346e+18     5158
Name: device_model, dtype: int64

In [11]:
installs['wifi'].value_counts()

True     235130
False     59699
Name: wifi, dtype: int64

In [12]:
installs['device_language'].value_counts().head(10)

6.977049e+18    233012
3.301378e+18    149638
8.441417e+18     46449
6.584214e+18      6041
4.060930e+18      5222
5.221863e+18      3867
1.774282e+18      2565
7.528974e+18      1778
3.491909e+18      1490
2.938044e+18       902
Name: device_language, dtype: int64

### Creo lo archivos de features de las instalaciones

In [23]:
def crear_features_installs(installs_entrenar,datetime_inicio):
    installs_entrenar['created'] = pd.to_datetime(installs_entrenar['created'])
    
    dia_en_segundos = 86400
    installs_entrenar['segundos'] = ((installs_entrenar['created']-datetime_inicio).dt.seconds + 
                                     (installs_entrenar['created']-datetime_inicio).dt.days*dia_en_segundos)

    
    # Cambio application_id por la probabilidad de que aparezca esa aplicacion
    aplicacion = installs_entrenar['application_id'].value_counts(normalize=True)
    installs_entrenar['application_id'] = installs_entrenar['application_id'].replace(aplicacion)
        
    # Cambio device_model por la probabilidad de que aparezca ese modelo
    modelo = installs_entrenar['device_model'].value_counts(normalize=True)
    installs_entrenar['device_model'] = installs_entrenar['device_model'].replace(modelo)
    
    # Cambio device_language por la probabilidad de que aparezca ese lenguaje
    lenguaje = installs_entrenar['device_language'].value_counts(normalize=True)
    installs_entrenar['device_language'] = installs_entrenar['device_language'].replace(lenguaje)
    
    # Cambio ref_type por la probabilidad de que aparezca ese ref_type
    tipo = installs_entrenar['ref_type'].value_counts(normalize=True)
    installs_entrenar['ref_type'] = installs_entrenar['ref_type'].replace(tipo)
    
    # Agrego los wifi que faltan
    installs_entrenar['wifi'] = installs_entrenar['wifi'].fillna(False)

    # Agrupo el data frame por los ref_hash
    installs_entrenar = installs_entrenar.groupby('ref_hash').agg({'ref_hash':'count','attributed':'sum',
                                                               'wifi':'sum','implicit':'sum',
                                                               'application_id':'mean','device_model':'mean',
                                                               'device_language':'mean','ref_type':'mean',
                                                               'segundos':['min','mean','max']})
    
    installs_entrenar.columns = ['i_t_min','i_t_prom','i_t_max','i_app_id','i_#installs',
                                 'i_model','i_language','i_type','i_wifi','i_implicit','i_attributed']
    installs_entrenar = installs_entrenar.reset_index('ref_hash')
          
    return installs_entrenar

In [24]:
installs_entrenar = pd.read_csv('installs_1820.csv')
datetime_inicio = datetime(2019,4,18)
f_installs = crear_features_installs(installs_entrenar,datetime_inicio)
f_installs.to_csv('installs_train_1820.csv', index=False)
f_installs.head()

,ref_hash,i_t_min,i_t_prom,i_t_max,i_app_id,i_#installs,i_model,i_language,i_type,i_wifi,i_implicit,i_attributed
0,41863526108385,76310,76472.0,76636,0.109299,4,0.006530,0.526228,0.834491,0.0,2.0,False
1,135153013040192,192980,192980.0,192980,0.026967,1,0.000039,0.316946,0.834491,0.0,0.0,False
2,186034136943920,60042,60042.0,60042,0.063763,1,NaN,NaN,0.834491,0.0,1.0,False
3,365882020742330,96156,96156.0,96156,0.014889,1,0.004079,0.526228,0.834491,0.0,0.0,False
4,519199987760489,92860,92860.0,92860,0.012377,1,0.001007,0.526228,0.834491,1.0,0.0,False


In [25]:
installs_entrenar = pd.read_csv('installs_1921.csv')
datetime_inicio = datetime(2019,4,19)
f_installs = crear_features_installs(installs_entrenar,datetime_inicio)
f_installs.to_csv('installs_train_1921.csv', index=False)
f_installs.head()

,ref_hash,i_t_min,i_t_prom,i_t_max,i_app_id,i_#installs,i_model,i_language,i_type,i_wifi,i_implicit,i_attributed
0,40621409780134,242267,242267.0,242267,0.011750,1,0.006618,0.317287,0.833182,0.0,0.0,0.0
1,135153013040192,106580,106580.0,106580,0.027300,1,0.000060,0.317287,0.833182,0.0,0.0,0.0
2,345999128501141,188378,188378.0,188378,0.027300,1,0.002233,0.317287,0.833182,0.0,0.0,0.0
3,365882020742330,9756,9756.0,9756,0.015855,1,0.003899,0.524056,0.833182,0.0,0.0,0.0
4,457867161681660,251847,251847.0,251847,0.040905,1,0.101014,0.102965,0.166818,1.0,0.0,0.0


In [26]:
installs_entrenar = pd.read_csv('installs_2022.csv')
datetime_inicio = datetime(2019,4,20)
f_installs = crear_features_installs(installs_entrenar,datetime_inicio)
f_installs.to_csv('installs_train_2022.csv', index=False)
f_installs.head()

,ref_hash,i_t_min,i_t_prom,i_t_max,i_app_id,i_#installs,i_model,i_language,i_type,i_wifi,i_implicit,i_attributed
0,40621409780134,155867,155867.0,155867,0.012172,1,0.006409,0.324184,0.828972,0.0,0.0,0.0
1,135153013040192,20180,20180.0,20180,0.028381,1,0.000060,0.324184,0.828972,0.0,0.0,0.0
2,161514654074162,238424,238424.0,238424,0.056471,1,0.007606,0.515034,0.828972,1.0,1.0,0.0
3,168103949904656,173373,173373.0,173373,0.028381,1,0.000253,0.324184,0.828972,0.0,0.0,0.0
4,283297668933729,222129,222129.0,222129,0.068694,1,0.003916,0.324184,0.828972,0.0,0.0,0.0


In [27]:
installs_entrenar = pd.read_csv('installs_2123.csv')
datetime_inicio = datetime(2019,4,21)
f_installs = crear_features_installs(installs_entrenar,datetime_inicio)
f_installs.to_csv('installs_train_2123.csv', index=False)
f_installs.head()

,ref_hash,i_t_min,i_t_prom,i_t_max,i_app_id,i_#installs,i_model,i_language,i_type,i_wifi,i_implicit,i_attributed
0,40621409780134,69467,69467.0,69467,0.012626,1,0.006177,0.328401,0.826917,0.0,0.0,0.0
1,161514654074162,152024,152024.0,152024,0.049540,1,0.007135,0.510374,0.826917,1.0,1.0,0.0
2,168103949904656,86973,86973.0,86973,0.028577,1,0.000251,0.328401,0.826917,0.0,0.0,0.0
3,283297668933729,135729,135729.0,135729,0.070329,1,0.003977,0.328401,0.826917,0.0,0.0,0.0
4,345999128501141,15578,15578.0,15578,0.028577,1,0.002233,0.328401,0.826917,0.0,0.0,0.0


In [28]:
installs_entrenar = pd.read_csv('installs_2426.csv')
datetime_inicio = datetime(2019,4,24)
f_installs = crear_features_installs(installs_entrenar,datetime_inicio)
f_installs.to_csv('installs_train_2426.csv', index=False)
f_installs.head()

,ref_hash,i_t_min,i_t_prom,i_t_max,i_app_id,i_#installs,i_model,i_language,i_type,i_wifi,i_implicit,i_attributed
0,90072729247980,66650,66650.0,66650,0.075728,1,0.001783,0.343916,0.823905,0.0,0.0,0.0
1,342614246084071,253421,253421.0,253421,0.005268,1,0.002016,0.343916,0.176095,0.0,0.0,0.0
2,347785260789835,250811,250811.0,250811,0.000562,1,0.000433,0.343916,0.823905,0.0,1.0,0.0
3,416301579449694,140951,140951.0,140951,0.000256,1,NaN,NaN,0.823905,0.0,0.0,0.0
4,420449720220692,147791,147791.0,147791,0.007042,1,NaN,NaN,0.823905,0.0,0.0,0.0


### Creo los archivos para testear las instalaciones

In [29]:
def crear_test_installs(installs_validar,datetime_validacion):
    
    dia_en_segundos = 86400
    installs_validar['created'] = pd.to_datetime(installs_validar['created'])
    
    installs_validar['segundos'] = ((installs_validar['created']-datetime_validacion).dt.seconds + 
                                     (installs_validar['created']-datetime_validacion).dt.days*dia_en_segundos)
        
    installs_validar = installs_validar.groupby('ref_hash').agg({'segundos':'mean'}).reset_index('ref_hash')  
        
    return installs_validar
   

In [30]:
installs_validar = pd.read_csv('installs_2123.csv')
datetime_validar = datetime(2019,4,21)
installs_validar = crear_test_installs(installs_validar,datetime_validar)
installs_validar.to_csv('installs_test_1820.csv', index=False)
installs_validar.head()

,ref_hash,segundos
0,40621409780134,69467
1,161514654074162,152024
2,168103949904656,86973
3,283297668933729,135729
4,345999128501141,15578


In [31]:
installs_validar = pd.read_csv('installs_2224.csv')
datetime_validar = datetime(2019,4,22)
installs_validar = crear_test_installs(installs_validar,datetime_validar)
installs_validar.to_csv('installs_test_1921.csv', index=False)

In [32]:
installs_validar = pd.read_csv('installs_2325.csv')
datetime_validar = datetime(2019,4,23)
installs_validar = crear_test_installs(installs_validar,datetime_validar)
installs_validar.to_csv('installs_test_2022.csv', index=False)

In [33]:
installs_validar = pd.read_csv('installs_2426.csv')
datetime_validar = datetime(2019,4,24)
installs_validar = crear_test_installs(installs_validar,datetime_validar)
installs_validar.to_csv('installs_test_2123.csv', index=False)